<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preparation" data-toc-modified-id="Preparation-1">Preparation</a></span></li><li><span><a href="#EfficientNetB0" data-toc-modified-id="EfficientNetB0-2">EfficientNetB0</a></span><ul class="toc-item"><li><span><a href="#224-with-augmentation" data-toc-modified-id="224-with-augmentation-2.1">224 with augmentation</a></span></li><li><span><a href="#224-without-augmentation" data-toc-modified-id="224-without-augmentation-2.2">224 without augmentation</a></span></li></ul></li><li><span><a href="#EfficientNetB1" data-toc-modified-id="EfficientNetB1-3">EfficientNetB1</a></span><ul class="toc-item"><li><span><a href="#224-with-augmentation" data-toc-modified-id="224-with-augmentation-3.1">224 with augmentation</a></span></li><li><span><a href="#224-without-augmentation" data-toc-modified-id="224-without-augmentation-3.2">224 without augmentation</a></span></li><li><span><a href="#240-with-augmentation" data-toc-modified-id="240-with-augmentation-3.3">240 with augmentation</a></span></li></ul></li><li><span><a href="#EfficientNetB2" data-toc-modified-id="EfficientNetB2-4">EfficientNetB2</a></span><ul class="toc-item"><li><span><a href="#224-with-augmentation" data-toc-modified-id="224-with-augmentation-4.1">224 with augmentation</a></span></li><li><span><a href="#224-without-augmentation" data-toc-modified-id="224-without-augmentation-4.2">224 without augmentation</a></span></li><li><span><a href="#260-with-augmentation" data-toc-modified-id="260-with-augmentation-4.3">260 with augmentation</a></span></li><li><span><a href="#260-without-aucmentation" data-toc-modified-id="260-without-aucmentation-4.4">260 without aucmentation</a></span></li></ul></li><li><span><a href="#Model-Ensemble" data-toc-modified-id="Model-Ensemble-5">Model Ensemble</a></span><ul class="toc-item"><li><span><a href="#3-models" data-toc-modified-id="3-models-5.1">3 models</a></span></li><li><span><a href="#4-models" data-toc-modified-id="4-models-5.2">4 models</a></span></li></ul></li></ul></div>

# Preparation

In [1]:
import os
import glob
import cv2
import tensorflow as tf
import numpy as np
import json
from tqdm.notebook import tqdm
import collections

In [2]:
base_dir = os.getenv("HOME") + "/UDIGO"
model_dir = base_dir + "/models"
test_dir = base_dir + "/dataset/test"
data_dir = base_dir + "/data"

In [3]:
os.listdir(model_dir)

['efn_b2_260_9-0.40-0.91.h5',
 'efn_b1_240_6-0.41-0.90.h5',
 'efn_b0_224_na_5-0.43-0.90.h5',
 'efn_b1_224_na_5-0.43-0.90.h5',
 'efn_b2_224_5-0.42-0.89.h5',
 'efn_b1_224_11-0.46-0.90.h5',
 'efn_b0_224_9-0.45-0.90.h5',
 'efn_b2_260_na_5-0.41-0.91.h5',
 'efn_b2_224_na_3-0.45-0.88.h5']

In [4]:
image_list = glob.glob(test_dir + "/*/*")
len(image_list)

10970

In [5]:
image_list[:10]

['/home/ssac21/UDIGO/dataset/test/산/349.jpg',
 '/home/ssac21/UDIGO/dataset/test/산/00002795.jpg',
 '/home/ssac21/UDIGO/dataset/test/산/178.jpg',
 '/home/ssac21/UDIGO/dataset/test/산/00003432.jpg',
 '/home/ssac21/UDIGO/dataset/test/산/67.jpg',
 '/home/ssac21/UDIGO/dataset/test/산/318.jpg',
 '/home/ssac21/UDIGO/dataset/test/산/00004403.jpg',
 '/home/ssac21/UDIGO/dataset/test/산/00002345.jpg',
 '/home/ssac21/UDIGO/dataset/test/산/108.jpg',
 '/home/ssac21/UDIGO/dataset/test/산/342.jpg']

In [6]:
with open(os.getenv("HOME") + "/UDIGO/data/place_55_label_to_id.json", "r") as f:
    label_dict = json.load(f)

In [7]:
def inference(img_path, img_size):
    image = cv2.imread(img_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (img_size, img_size))
    image = image[np.newaxis, :, :, :]
    pred = model.predict(image, batch_size=1)
    return np.argmax(pred)

In [8]:
def test_model(image_list, img_size):
    label_list = []
    correct_dict = {}
    correct = 0
    n = 0

    for image in tqdm(image_list):
        try:
            pred = inference(image, img_size)
            n += 1

            label = image.split("/")[-2]
            label_list.append(label)
            if label not in correct_dict:
                correct_dict[label] = 0

            label_encoding = label_dict[label]
            if pred == label_encoding:
                correct_dict[label] += 1
                correct += 1

        except:
            print("Invalid image:", image)
            continue

    print(f"{n} data, test complete!")
    print("Test Accuracy:", round(correct / n, 3))
    return label_list, correct_dict

# EfficientNetB0

## 224 with augmentation

In [7]:
model = tf.keras.models.load_model(os.path.join(model_dir, 'efn_b0_224_9-0.45-0.90.h5'))

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 224, 224, 3)       0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 1280)              4049571   
_________________________________________________________________
dense (Dense)                (None, 55)                70455     
Total params: 4,120,026
Trainable params: 4,078,003
Non-trainable params: 42,023
_________________________________________________________________


In [12]:
label_list, correct_dict = test_model(image_list, img_size=224)

  0%|          | 0/10970 [00:00<?, ?it/s]

Invalid image: /home/ssac21/UDIGO/dataset/test/수영장/img03_01_02_04.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/one-footed_skiing064.gif?type=w580
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/surf_board01.gif?type=w2
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf2698b4fe5dfcb1ac9187aa34c06699.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf42ca3213a39ecdf97ed5141b23bc71.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_580
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_001.gif?type=w1
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/095162b2-e684-4a65-a8de-48a93307a596.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/a4b6d053-877c-48db-a5a1-dc808ea6db46.gif
10961 data, test complete!
Test Accuracy: 0.895


In [13]:
count = collections.Counter(label_list)
count

Counter({'산': 200,
         '청와대': 200,
         '농구장': 200,
         '시장': 200,
         '수영장': 199,
         '미용실': 200,
         '수상레포츠': 198,
         '놀이공원': 200,
         '야구장': 198,
         '미술관': 199,
         '클럽': 200,
         '호텔': 200,
         '식물원': 199,
         '키즈카페': 200,
         '빨래방': 200,
         '대학교': 200,
         '병원': 200,
         '박물관': 196,
         '한옥마을': 200,
         '골프장': 200,
         '동물원': 200,
         '숲': 194,
         '아이스링크': 197,
         '동대문디자인플라자': 200,
         '지하철역': 200,
         '세빛섬': 200,
         '폭포': 198,
         '놀이터': 200,
         '아쿠아리움': 200,
         '절': 200,
         '다리': 200,
         'PC방': 200,
         '쇼핑몰': 200,
         '63빌딩': 200,
         '컨벤션센터': 200,
         '국회의사당': 200,
         '공원': 197,
         '롯데월드타워': 200,
         '헬스장': 200,
         '공항': 195,
         '교회': 200,
         '궁궐': 200,
         '도서관': 200,
         '워터파크': 198,
         '축구장': 200,
         '성당': 200,
         '캠핑장': 200,
     

In [14]:
correct_dict

{'산': 190,
 '청와대': 198,
 '농구장': 197,
 '시장': 185,
 '수영장': 183,
 '미용실': 166,
 '수상레포츠': 188,
 '놀이공원': 134,
 '야구장': 190,
 '미술관': 142,
 '클럽': 123,
 '호텔': 190,
 '식물원': 189,
 '키즈카페': 171,
 '빨래방': 188,
 '대학교': 161,
 '병원': 155,
 '박물관': 147,
 '한옥마을': 153,
 '골프장': 168,
 '동물원': 189,
 '숲': 183,
 '아이스링크': 181,
 '동대문디자인플라자': 196,
 '지하철역': 188,
 '세빛섬': 196,
 '폭포': 195,
 '놀이터': 171,
 '아쿠아리움': 186,
 '절': 140,
 '다리': 155,
 'PC방': 196,
 '쇼핑몰': 182,
 '63빌딩': 192,
 '컨벤션센터': 183,
 '국회의사당': 196,
 '공원': 163,
 '롯데월드타워': 172,
 '헬스장': 180,
 '공항': 172,
 '교회': 189,
 '궁궐': 190,
 '도서관': 191,
 '워터파크': 135,
 '축구장': 191,
 '성당': 176,
 '캠핑장': 186,
 '볼링장': 192,
 '패스트푸드점': 186,
 '베이커리': 189,
 '공연장': 183,
 '해수욕장': 189,
 '스키장': 196,
 '찜질방': 188,
 '남산서울타워': 195}

In [15]:
accuracy_dict = {}
for label in correct_dict:
    accuracy_dict[label] = round(correct_dict[label]/count[label], 3)

sorted(accuracy_dict.items(), key=lambda x: x[1], reverse=True)

[('청와대', 0.99),
 ('농구장', 0.985),
 ('폭포', 0.985),
 ('동대문디자인플라자', 0.98),
 ('세빛섬', 0.98),
 ('PC방', 0.98),
 ('국회의사당', 0.98),
 ('스키장', 0.98),
 ('남산서울타워', 0.975),
 ('볼링장', 0.97),
 ('야구장', 0.96),
 ('63빌딩', 0.96),
 ('도서관', 0.955),
 ('축구장', 0.955),
 ('산', 0.95),
 ('호텔', 0.95),
 ('식물원', 0.95),
 ('궁궐', 0.95),
 ('수상레포츠', 0.949),
 ('동물원', 0.945),
 ('교회', 0.945),
 ('베이커리', 0.945),
 ('해수욕장', 0.945),
 ('숲', 0.943),
 ('빨래방', 0.94),
 ('지하철역', 0.94),
 ('찜질방', 0.94),
 ('공연장', 0.938),
 ('아쿠아리움', 0.93),
 ('캠핑장', 0.93),
 ('패스트푸드점', 0.93),
 ('시장', 0.925),
 ('수영장', 0.92),
 ('아이스링크', 0.919),
 ('컨벤션센터', 0.915),
 ('쇼핑몰', 0.91),
 ('헬스장', 0.9),
 ('공항', 0.882),
 ('성당', 0.88),
 ('롯데월드타워', 0.86),
 ('키즈카페', 0.855),
 ('놀이터', 0.855),
 ('골프장', 0.84),
 ('미용실', 0.83),
 ('공원', 0.827),
 ('대학교', 0.805),
 ('병원', 0.775),
 ('다리', 0.775),
 ('한옥마을', 0.765),
 ('박물관', 0.75),
 ('미술관', 0.714),
 ('절', 0.7),
 ('워터파크', 0.682),
 ('놀이공원', 0.67),
 ('클럽', 0.615)]

## 224 without augmentation

In [9]:
model = tf.keras.models.load_model(os.path.join(model_dir, 'efn_b0_224_na_5-0.43-0.90.h5'))

In [10]:
label_list, correct_dict = test_model(image_list, img_size=224)

  0%|          | 0/10970 [00:00<?, ?it/s]

Invalid image: /home/ssac21/UDIGO/dataset/test/수영장/img03_01_02_04.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/one-footed_skiing064.gif?type=w580
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/surf_board01.gif?type=w2
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf2698b4fe5dfcb1ac9187aa34c06699.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf42ca3213a39ecdf97ed5141b23bc71.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_580
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_001.gif?type=w1
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/095162b2-e684-4a65-a8de-48a93307a596.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/a4b6d053-877c-48db-a5a1-dc808ea6db46.gif
10961 data, test complete!
Test Accuracy: 0.905


In [11]:
count = collections.Counter(label_list)
accuracy_dict = {}
for label in correct_dict:
    accuracy_dict[label] = round(correct_dict[label]/count[label], 3)

sorted(accuracy_dict.items(), key=lambda x: x[1], reverse=True)

[('세빛섬', 1.0),
 ('청와대', 0.995),
 ('폭포', 0.995),
 ('농구장', 0.99),
 ('국회의사당', 0.99),
 ('동대문디자인플라자', 0.985),
 ('시장', 0.98),
 ('PC방', 0.98),
 ('교회', 0.98),
 ('축구장', 0.975),
 ('남산서울타워', 0.975),
 ('호텔', 0.97),
 ('동물원', 0.97),
 ('아쿠아리움', 0.97),
 ('식물원', 0.965),
 ('빨래방', 0.965),
 ('지하철역', 0.965),
 ('63빌딩', 0.96),
 ('도서관', 0.96),
 ('볼링장', 0.96),
 ('스키장', 0.96),
 ('컨벤션센터', 0.955),
 ('산', 0.95),
 ('야구장', 0.949),
 ('베이커리', 0.945),
 ('찜질방', 0.945),
 ('캠핑장', 0.94),
 ('패스트푸드점', 0.935),
 ('헬스장', 0.93),
 ('궁궐', 0.93),
 ('해수욕장', 0.93),
 ('수상레포츠', 0.929),
 ('숲', 0.923),
 ('공연장', 0.918),
 ('수영장', 0.91),
 ('쇼핑몰', 0.895),
 ('아이스링크', 0.888),
 ('미용실', 0.875),
 ('성당', 0.875),
 ('골프장', 0.865),
 ('놀이터', 0.865),
 ('롯데월드타워', 0.86),
 ('공항', 0.856),
 ('대학교', 0.845),
 ('다리', 0.815),
 ('키즈카페', 0.805),
 ('한옥마을', 0.785),
 ('절', 0.78),
 ('공원', 0.777),
 ('박물관', 0.77),
 ('병원', 0.75),
 ('미술관', 0.739),
 ('클럽', 0.725),
 ('워터파크', 0.722),
 ('놀이공원', 0.71)]

# EfficientNetB1

## 224 with augmentation

In [17]:
model = tf.keras.models.load_model(os.path.join(model_dir, 'efn_b1_224_11-0.46-0.90.h5'))

In [18]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 224, 224, 3)       0         
_________________________________________________________________
efficientnetb1 (Functional)  (None, 1280)              6575239   
_________________________________________________________________
dense (Dense)                (None, 55)                70455     
Total params: 6,645,694
Trainable params: 6,583,639
Non-trainable params: 62,055
_________________________________________________________________


In [19]:
label_list, correct_dict = test_model(image_list, img_size=224)

  0%|          | 0/10970 [00:00<?, ?it/s]

Invalid image: /home/ssac21/UDIGO/dataset/test/수영장/img03_01_02_04.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/one-footed_skiing064.gif?type=w580
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/surf_board01.gif?type=w2
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf2698b4fe5dfcb1ac9187aa34c06699.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf42ca3213a39ecdf97ed5141b23bc71.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_580
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_001.gif?type=w1
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/095162b2-e684-4a65-a8de-48a93307a596.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/a4b6d053-877c-48db-a5a1-dc808ea6db46.gif
10961 data, test complete!
Test Accuracy: 0.902


In [20]:
count = collections.Counter(label_list)
accuracy_dict = {}
for label in correct_dict:
    accuracy_dict[label] = round(correct_dict[label]/count[label], 3)

sorted(accuracy_dict.items(), key=lambda x: x[1], reverse=True)

[('청와대', 0.995),
 ('세빛섬', 0.99),
 ('폭포', 0.99),
 ('농구장', 0.985),
 ('국회의사당', 0.985),
 ('시장', 0.98),
 ('호텔', 0.98),
 ('동대문디자인플라자', 0.98),
 ('교회', 0.98),
 ('수상레포츠', 0.975),
 ('동물원', 0.97),
 ('PC방', 0.97),
 ('컨벤션센터', 0.97),
 ('지하철역', 0.965),
 ('남산서울타워', 0.965),
 ('축구장', 0.96),
 ('스키장', 0.96),
 ('야구장', 0.955),
 ('63빌딩', 0.955),
 ('볼링장', 0.955),
 ('식물원', 0.95),
 ('빨래방', 0.95),
 ('산', 0.945),
 ('수영장', 0.945),
 ('도서관', 0.945),
 ('숲', 0.943),
 ('궁궐', 0.94),
 ('패스트푸드점', 0.94),
 ('베이커리', 0.94),
 ('해수욕장', 0.935),
 ('아쿠아리움', 0.93),
 ('캠핑장', 0.93),
 ('찜질방', 0.91),
 ('공연장', 0.908),
 ('헬스장', 0.905),
 ('골프장', 0.9),
 ('아이스링크', 0.898),
 ('미용실', 0.88),
 ('성당', 0.88),
 ('놀이터', 0.87),
 ('롯데월드타워', 0.865),
 ('쇼핑몰', 0.86),
 ('공항', 0.856),
 ('키즈카페', 0.835),
 ('대학교', 0.815),
 ('절', 0.795),
 ('한옥마을', 0.78),
 ('워터파크', 0.778),
 ('다리', 0.77),
 ('미술관', 0.759),
 ('공원', 0.756),
 ('박물관', 0.755),
 ('병원', 0.75),
 ('클럽', 0.675),
 ('놀이공원', 0.655)]

## 224 without augmentation

In [9]:
model = tf.keras.models.load_model(os.path.join(model_dir, 'efn_b1_224_na_5-0.43-0.90.h5'))

In [10]:
label_list, correct_dict = test_model(image_list, img_size=224)

  0%|          | 0/10970 [00:00<?, ?it/s]

Invalid image: /home/ssac21/UDIGO/dataset/test/수영장/img03_01_02_04.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/one-footed_skiing064.gif?type=w580
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/surf_board01.gif?type=w2
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf2698b4fe5dfcb1ac9187aa34c06699.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf42ca3213a39ecdf97ed5141b23bc71.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_580
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_001.gif?type=w1
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/095162b2-e684-4a65-a8de-48a93307a596.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/a4b6d053-877c-48db-a5a1-dc808ea6db46.gif
10961 data, test complete!
Test Accuracy: 0.901


In [11]:
count = collections.Counter(label_list)
accuracy_dict = {}
for label in correct_dict:
    accuracy_dict[label] = round(correct_dict[label]/count[label], 3)

sorted(accuracy_dict.items(), key=lambda x: x[1], reverse=True)

[('청와대', 0.995),
 ('세빛섬', 0.995),
 ('폭포', 0.99),
 ('국회의사당', 0.99),
 ('농구장', 0.985),
 ('축구장', 0.98),
 ('산', 0.975),
 ('동대문디자인플라자', 0.975),
 ('PC방', 0.975),
 ('교회', 0.975),
 ('시장', 0.97),
 ('도서관', 0.97),
 ('호텔', 0.965),
 ('동물원', 0.965),
 ('패스트푸드점', 0.965),
 ('남산서울타워', 0.965),
 ('야구장', 0.96),
 ('지하철역', 0.96),
 ('63빌딩', 0.96),
 ('스키장', 0.96),
 ('식물원', 0.955),
 ('볼링장', 0.955),
 ('숲', 0.954),
 ('아쿠아리움', 0.945),
 ('궁궐', 0.945),
 ('수상레포츠', 0.939),
 ('빨래방', 0.935),
 ('컨벤션센터', 0.93),
 ('캠핑장', 0.92),
 ('베이커리', 0.92),
 ('해수욕장', 0.92),
 ('수영장', 0.915),
 ('공항', 0.913),
 ('찜질방', 0.91),
 ('공연장', 0.903),
 ('헬스장', 0.9),
 ('미용실', 0.895),
 ('아이스링크', 0.883),
 ('골프장', 0.875),
 ('쇼핑몰', 0.87),
 ('성당', 0.87),
 ('놀이터', 0.865),
 ('롯데월드타워', 0.85),
 ('다리', 0.82),
 ('공원', 0.797),
 ('키즈카페', 0.795),
 ('대학교', 0.795),
 ('한옥마을', 0.795),
 ('박물관', 0.776),
 ('병원', 0.755),
 ('워터파크', 0.747),
 ('절', 0.735),
 ('미술관', 0.724),
 ('클럽', 0.72),
 ('놀이공원', 0.68)]

## 240 with augmentation

In [12]:
model = tf.keras.models.load_model(os.path.join(model_dir, 'efn_b1_240_6-0.41-0.90.h5'))

In [13]:
label_list, correct_dict = test_model(image_list, img_size=240)

  0%|          | 0/10970 [00:00<?, ?it/s]

Invalid image: /home/ssac21/UDIGO/dataset/test/수영장/img03_01_02_04.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/one-footed_skiing064.gif?type=w580
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/surf_board01.gif?type=w2
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf2698b4fe5dfcb1ac9187aa34c06699.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf42ca3213a39ecdf97ed5141b23bc71.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_580
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_001.gif?type=w1
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/095162b2-e684-4a65-a8de-48a93307a596.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/a4b6d053-877c-48db-a5a1-dc808ea6db46.gif
10961 data, test complete!
Test Accuracy: 0.889


In [14]:
count = collections.Counter(label_list)
accuracy_dict = {}
for label in correct_dict:
    accuracy_dict[label] = round(correct_dict[label]/count[label], 3)

sorted(accuracy_dict.items(), key=lambda x: x[1], reverse=True)

[('세빛섬', 0.995),
 ('폭포', 0.995),
 ('호텔', 0.99),
 ('국회의사당', 0.99),
 ('농구장', 0.985),
 ('동대문디자인플라자', 0.985),
 ('숲', 0.979),
 ('청와대', 0.975),
 ('PC방', 0.975),
 ('수상레포츠', 0.97),
 ('볼링장', 0.97),
 ('남산서울타워', 0.97),
 ('63빌딩', 0.96),
 ('축구장', 0.96),
 ('빨래방', 0.955),
 ('지하철역', 0.95),
 ('도서관', 0.95),
 ('스키장', 0.95),
 ('야구장', 0.949),
 ('식물원', 0.94),
 ('궁궐', 0.94),
 ('산', 0.93),
 ('찜질방', 0.93),
 ('동물원', 0.925),
 ('교회', 0.925),
 ('캠핑장', 0.92),
 ('패스트푸드점', 0.915),
 ('아이스링크', 0.914),
 ('헬스장', 0.91),
 ('베이커리', 0.91),
 ('해수욕장', 0.905),
 ('아쿠아리움', 0.9),
 ('컨벤션센터', 0.9),
 ('수영장', 0.899),
 ('공연장', 0.897),
 ('성당', 0.89),
 ('골프장', 0.88),
 ('놀이터', 0.88),
 ('쇼핑몰', 0.87),
 ('미용실', 0.855),
 ('시장', 0.85),
 ('키즈카페', 0.85),
 ('롯데월드타워', 0.835),
 ('공항', 0.826),
 ('다리', 0.81),
 ('대학교', 0.8),
 ('공원', 0.792),
 ('한옥마을', 0.785),
 ('병원', 0.77),
 ('미술관', 0.754),
 ('박물관', 0.75),
 ('워터파크', 0.672),
 ('클럽', 0.655),
 ('절', 0.655),
 ('놀이공원', 0.62)]

# EfficientNetB2

## 224 with augmentation

In [21]:
model = tf.keras.models.load_model(os.path.join(model_dir, 'efn_b2_224_5-0.42-0.89.h5'))

In [22]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 224, 224, 3)       0         
_________________________________________________________________
efficientnetb2 (Functional)  (None, 1408)              7768569   
_________________________________________________________________
dense (Dense)                (None, 55)                77495     
Total params: 7,846,064
Trainable params: 7,778,489
Non-trainable params: 67,575
_________________________________________________________________


In [23]:
label_list, correct_dict = test_model(image_list, img_size=224)

  0%|          | 0/10970 [00:00<?, ?it/s]

Invalid image: /home/ssac21/UDIGO/dataset/test/수영장/img03_01_02_04.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/one-footed_skiing064.gif?type=w580
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/surf_board01.gif?type=w2
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf2698b4fe5dfcb1ac9187aa34c06699.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf42ca3213a39ecdf97ed5141b23bc71.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_580
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_001.gif?type=w1
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/095162b2-e684-4a65-a8de-48a93307a596.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/a4b6d053-877c-48db-a5a1-dc808ea6db46.gif
10961 data, test complete!
Test Accuracy: 0.902


In [24]:
count = collections.Counter(label_list)
accuracy_dict = {}
for label in correct_dict:
    accuracy_dict[label] = round(correct_dict[label]/count[label], 3)

sorted(accuracy_dict.items(), key=lambda x: x[1], reverse=True)

[('청와대', 0.995),
 ('세빛섬', 0.995),
 ('농구장', 0.985),
 ('PC방', 0.985),
 ('국회의사당', 0.985),
 ('축구장', 0.985),
 ('동대문디자인플라자', 0.98),
 ('폭포', 0.98),
 ('교회', 0.98),
 ('호텔', 0.975),
 ('스키장', 0.975),
 ('동물원', 0.97),
 ('63빌딩', 0.97),
 ('수상레포츠', 0.965),
 ('지하철역', 0.965),
 ('남산서울타워', 0.965),
 ('야구장', 0.96),
 ('식물원', 0.96),
 ('컨벤션센터', 0.96),
 ('도서관', 0.96),
 ('캠핑장', 0.96),
 ('패스트푸드점', 0.96),
 ('볼링장', 0.955),
 ('베이커리', 0.95),
 ('아쿠아리움', 0.945),
 ('시장', 0.94),
 ('빨래방', 0.94),
 ('찜질방', 0.94),
 ('공연장', 0.933),
 ('미용실', 0.925),
 ('궁궐', 0.925),
 ('산', 0.92),
 ('숲', 0.918),
 ('수영장', 0.91),
 ('아이스링크', 0.904),
 ('헬스장', 0.895),
 ('해수욕장', 0.895),
 ('롯데월드타워', 0.89),
 ('성당', 0.88),
 ('놀이터', 0.875),
 ('골프장', 0.865),
 ('공항', 0.851),
 ('쇼핑몰', 0.845),
 ('대학교', 0.835),
 ('다리', 0.81),
 ('공원', 0.807),
 ('키즈카페', 0.805),
 ('한옥마을', 0.79),
 ('박물관', 0.786),
 ('절', 0.77),
 ('미술관', 0.749),
 ('워터파크', 0.737),
 ('병원', 0.715),
 ('클럽', 0.66),
 ('놀이공원', 0.655)]

## 224 without augmentation

In [12]:
model = tf.keras.models.load_model(os.path.join(model_dir, 'efn_b2_224_na_3-0.45-0.88.h5'))

In [13]:
label_list, correct_dict = test_model(image_list, img_size=224)

  0%|          | 0/10970 [00:00<?, ?it/s]

Invalid image: /home/ssac21/UDIGO/dataset/test/수영장/img03_01_02_04.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/one-footed_skiing064.gif?type=w580
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/surf_board01.gif?type=w2
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf2698b4fe5dfcb1ac9187aa34c06699.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf42ca3213a39ecdf97ed5141b23bc71.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_580
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_001.gif?type=w1
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/095162b2-e684-4a65-a8de-48a93307a596.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/a4b6d053-877c-48db-a5a1-dc808ea6db46.gif
10961 data, test complete!
Test Accuracy: 0.889


In [14]:
count = collections.Counter(label_list)
accuracy_dict = {}
for label in correct_dict:
    accuracy_dict[label] = round(correct_dict[label]/count[label], 3)

sorted(accuracy_dict.items(), key=lambda x: x[1], reverse=True)

[('폭포', 0.995),
 ('국회의사당', 0.995),
 ('세빛섬', 0.99),
 ('청와대', 0.985),
 ('PC방', 0.985),
 ('농구장', 0.98),
 ('시장', 0.98),
 ('동물원', 0.975),
 ('동대문디자인플라자', 0.97),
 ('63빌딩', 0.97),
 ('남산서울타워', 0.965),
 ('컨벤션센터', 0.96),
 ('야구장', 0.955),
 ('호텔', 0.955),
 ('축구장', 0.955),
 ('볼링장', 0.955),
 ('지하철역', 0.95),
 ('수상레포츠', 0.949),
 ('교회', 0.945),
 ('패스트푸드점', 0.94),
 ('식물원', 0.935),
 ('아쿠아리움', 0.935),
 ('스키장', 0.935),
 ('도서관', 0.93),
 ('빨래방', 0.925),
 ('헬스장', 0.925),
 ('숲', 0.923),
 ('산', 0.92),
 ('베이커리', 0.915),
 ('공연장', 0.908),
 ('궁궐', 0.905),
 ('캠핑장', 0.905),
 ('수영장', 0.899),
 ('아이스링크', 0.893),
 ('놀이터', 0.89),
 ('찜질방', 0.89),
 ('해수욕장', 0.885),
 ('미용실', 0.88),
 ('성당', 0.865),
 ('골프장', 0.845),
 ('키즈카페', 0.84),
 ('절', 0.835),
 ('쇼핑몰', 0.83),
 ('대학교', 0.825),
 ('다리', 0.815),
 ('공항', 0.81),
 ('롯데월드타워', 0.77),
 ('박물관', 0.76),
 ('한옥마을', 0.76),
 ('병원', 0.75),
 ('워터파크', 0.727),
 ('공원', 0.721),
 ('클럽', 0.685),
 ('놀이공원', 0.67),
 ('미술관', 0.648)]

## 260 with augmentation

In [25]:
model = tf.keras.models.load_model(os.path.join(model_dir, 'efn_b2_260_9-0.40-0.91.h5'))

In [27]:
label_list, correct_dict = test_model(image_list, img_size=260)

  0%|          | 0/10970 [00:00<?, ?it/s]

Invalid image: /home/ssac21/UDIGO/dataset/test/수영장/img03_01_02_04.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/one-footed_skiing064.gif?type=w580
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/surf_board01.gif?type=w2
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf2698b4fe5dfcb1ac9187aa34c06699.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf42ca3213a39ecdf97ed5141b23bc71.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_580
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_001.gif?type=w1
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/095162b2-e684-4a65-a8de-48a93307a596.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/a4b6d053-877c-48db-a5a1-dc808ea6db46.gif
10961 data, test complete!
Test Accuracy: 0.896


In [28]:
count = collections.Counter(label_list)
accuracy_dict = {}
for label in correct_dict:
    accuracy_dict[label] = round(correct_dict[label]/count[label], 3)

sorted(accuracy_dict.items(), key=lambda x: x[1], reverse=True)

[('세빛섬', 0.995),
 ('청와대', 0.99),
 ('농구장', 0.99),
 ('동대문디자인플라자', 0.985),
 ('호텔', 0.98),
 ('폭포', 0.98),
 ('국회의사당', 0.98),
 ('축구장', 0.98),
 ('남산서울타워', 0.98),
 ('PC방', 0.975),
 ('수상레포츠', 0.97),
 ('63빌딩', 0.97),
 ('도서관', 0.97),
 ('볼링장', 0.97),
 ('지하철역', 0.965),
 ('산', 0.96),
 ('야구장', 0.96),
 ('교회', 0.96),
 ('동물원', 0.95),
 ('숲', 0.943),
 ('빨래방', 0.94),
 ('궁궐', 0.94),
 ('패스트푸드점', 0.94),
 ('아이스링크', 0.939),
 ('수영장', 0.935),
 ('스키장', 0.93),
 ('캠핑장', 0.925),
 ('해수욕장', 0.925),
 ('찜질방', 0.925),
 ('식물원', 0.92),
 ('헬스장', 0.92),
 ('베이커리', 0.92),
 ('공연장', 0.913),
 ('아쿠아리움', 0.9),
 ('컨벤션센터', 0.895),
 ('골프장', 0.885),
 ('키즈카페', 0.875),
 ('미용실', 0.865),
 ('성당', 0.865),
 ('공항', 0.862),
 ('놀이터', 0.86),
 ('롯데월드타워', 0.855),
 ('쇼핑몰', 0.845),
 ('대학교', 0.84),
 ('시장', 0.835),
 ('한옥마을', 0.81),
 ('박물관', 0.796),
 ('미술관', 0.784),
 ('공원', 0.761),
 ('다리', 0.755),
 ('워터파크', 0.727),
 ('클럽', 0.71),
 ('병원', 0.695),
 ('놀이공원', 0.685),
 ('절', 0.665)]

## 260 without aucmentation

In [29]:
model = tf.keras.models.load_model(os.path.join(model_dir, 'efn_b2_260_na_5-0.41-0.91.h5'))

In [31]:
label_list, correct_dict = test_model(image_list, img_size=260)

  0%|          | 0/10970 [00:00<?, ?it/s]

Invalid image: /home/ssac21/UDIGO/dataset/test/수영장/img03_01_02_04.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/one-footed_skiing064.gif?type=w580
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/surf_board01.gif?type=w2
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf2698b4fe5dfcb1ac9187aa34c06699.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf42ca3213a39ecdf97ed5141b23bc71.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_580
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_001.gif?type=w1
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/095162b2-e684-4a65-a8de-48a93307a596.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/a4b6d053-877c-48db-a5a1-dc808ea6db46.gif
10961 data, test complete!
Test Accuracy: 0.908


In [32]:
count = collections.Counter(label_list)
accuracy_dict = {}
for label in correct_dict:
    accuracy_dict[label] = round(correct_dict[label]/count[label], 3)

sorted(accuracy_dict.items(), key=lambda x: x[1], reverse=True)

[('농구장', 1.0),
 ('세빛섬', 0.995),
 ('청와대', 0.99),
 ('폭포', 0.99),
 ('국회의사당', 0.99),
 ('PC방', 0.985),
 ('도서관', 0.985),
 ('동대문디자인플라자', 0.98),
 ('지하철역', 0.98),
 ('야구장', 0.975),
 ('남산서울타워', 0.975),
 ('호텔', 0.97),
 ('동물원', 0.97),
 ('교회', 0.97),
 ('축구장', 0.97),
 ('63빌딩', 0.965),
 ('볼링장', 0.965),
 ('스키장', 0.965),
 ('산', 0.96),
 ('수상레포츠', 0.955),
 ('컨벤션센터', 0.95),
 ('궁궐', 0.95),
 ('캠핑장', 0.95),
 ('시장', 0.945),
 ('식물원', 0.945),
 ('빨래방', 0.945),
 ('패스트푸드점', 0.945),
 ('아쿠아리움', 0.94),
 ('베이커리', 0.935),
 ('찜질방', 0.935),
 ('아이스링크', 0.934),
 ('숲', 0.928),
 ('공연장', 0.928),
 ('수영장', 0.915),
 ('헬스장', 0.905),
 ('해수욕장', 0.9),
 ('성당', 0.895),
 ('미용실', 0.89),
 ('골프장', 0.89),
 ('놀이터', 0.875),
 ('쇼핑몰', 0.875),
 ('롯데월드타워', 0.875),
 ('공항', 0.867),
 ('키즈카페', 0.855),
 ('공원', 0.848),
 ('다리', 0.84),
 ('대학교', 0.83),
 ('한옥마을', 0.805),
 ('박물관', 0.786),
 ('병원', 0.775),
 ('워터파크', 0.737),
 ('미술관', 0.734),
 ('놀이공원', 0.7),
 ('클럽', 0.7),
 ('절', 0.7)]

# Model Ensemble

## 3 models

In [7]:
def inference_ensemble(img_path):
    image = cv2.imread(img_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    image = image[np.newaxis, :, :, :]
    
    prediction = []
    for model in models:
        pred = model.predict(image, batch_size=1)
        prediction.append(np.argmax(pred))
    
    vote_for_ensemble = collections.Counter(prediction).most_common()[0][0]
    return vote_for_ensemble

In [8]:
def test_model_ensemble(image_list):
    label_list = []
    correct_dict = {}
    correct = 0
    n = 0

    for image in tqdm(image_list):
        try:
            pred = inference_ensemble(image)
            n += 1

            label = image.split("/")[-2]
            label_list.append(label)
            if label not in correct_dict:
                correct_dict[label] = 0

            label_encoding = label_dict[label]
            if pred == label_encoding:
                correct_dict[label] += 1
                correct += 1

        except:
            print("Invalid image:", image)
            continue

    print(f"{n} data, test complete!")
    print("Test Accuracy:", round(correct / n, 3))
    return label_list, correct_dict

In [9]:
os.listdir(model_dir)

['efn_b2_260_9-0.40-0.91.h5',
 'efn_b1_240_6-0.41-0.90.h5',
 'efn_b0_224_na_5-0.43-0.90.h5',
 'efn_b1_224_na_5-0.43-0.90.h5',
 'efn_b2_224_5-0.42-0.89.h5',
 'efn_b1_224_11-0.46-0.90.h5',
 'efn_b0_224_9-0.45-0.90.h5',
 'efn_b2_260_na_5-0.41-0.91.h5',
 'efn_b2_224_na_3-0.45-0.88.h5']

In [10]:
model_1 = tf.keras.models.load_model(os.path.join(model_dir,  'efn_b0_224_na_5-0.43-0.90.h5'))
model_2 = tf.keras.models.load_model(os.path.join(model_dir, 'efn_b1_224_11-0.46-0.90.h5'))
model_3 = tf.keras.models.load_model(os.path.join(model_dir, 'efn_b2_224_5-0.42-0.89.h5'))
models = [model_1, model_2, model_3]

In [11]:
label_list, correct_dict = test_model_ensemble(image_list)

  0%|          | 0/10970 [00:00<?, ?it/s]

Invalid image: /home/ssac21/UDIGO/dataset/test/수영장/img03_01_02_04.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/one-footed_skiing064.gif?type=w580
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/surf_board01.gif?type=w2
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf2698b4fe5dfcb1ac9187aa34c06699.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf42ca3213a39ecdf97ed5141b23bc71.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_580
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_001.gif?type=w1
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/095162b2-e684-4a65-a8de-48a93307a596.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/a4b6d053-877c-48db-a5a1-dc808ea6db46.gif
10961 data, test complete!
Test Accuracy: 0.918


In [12]:
count = collections.Counter(label_list)
accuracy_dict = {}

for label in correct_dict:
    accuracy_dict[label] = round(correct_dict[label]/count[label], 3)

sorted(accuracy_dict.items(), key=lambda x: x[1], reverse=True)

[('세빛섬', 1.0),
 ('청와대', 0.995),
 ('폭포', 0.995),
 ('국회의사당', 0.995),
 ('농구장', 0.99),
 ('호텔', 0.985),
 ('동대문디자인플라자', 0.985),
 ('교회', 0.985),
 ('축구장', 0.985),
 ('식물원', 0.98),
 ('동물원', 0.98),
 ('PC방', 0.98),
 ('스키장', 0.98),
 ('시장', 0.975),
 ('남산서울타워', 0.975),
 ('수상레포츠', 0.97),
 ('지하철역', 0.97),
 ('컨벤션센터', 0.97),
 ('캠핑장', 0.97),
 ('패스트푸드점', 0.97),
 ('63빌딩', 0.965),
 ('도서관', 0.965),
 ('볼링장', 0.965),
 ('빨래방', 0.96),
 ('아쿠아리움', 0.96),
 ('베이커리', 0.96),
 ('야구장', 0.955),
 ('산', 0.945),
 ('찜질방', 0.945),
 ('궁궐', 0.94),
 ('공연장', 0.938),
 ('헬스장', 0.935),
 ('해수욕장', 0.935),
 ('숲', 0.933),
 ('아이스링크', 0.924),
 ('수영장', 0.92),
 ('미용실', 0.915),
 ('공항', 0.897),
 ('골프장', 0.89),
 ('쇼핑몰', 0.89),
 ('성당', 0.89),
 ('놀이터', 0.885),
 ('롯데월드타워', 0.885),
 ('대학교', 0.85),
 ('키즈카페', 0.845),
 ('다리', 0.845),
 ('절', 0.82),
 ('공원', 0.817),
 ('한옥마을', 0.815),
 ('박물관', 0.796),
 ('미술관', 0.784),
 ('워터파크', 0.753),
 ('병원', 0.745),
 ('놀이공원', 0.71),
 ('클럽', 0.695)]

## 4 models

In [9]:
model_1 = tf.keras.models.load_model(os.path.join(model_dir,  'efn_b0_224_na_5-0.43-0.90.h5'))
model_2 = tf.keras.models.load_model(os.path.join(model_dir, 'efn_b1_224_11-0.46-0.90.h5'))
model_3 = tf.keras.models.load_model(os.path.join(model_dir, 'efn_b2_224_5-0.42-0.89.h5'))
model_4 = tf.keras.models.load_model(os.path.join(model_dir, 'efn_b1_224_na_5-0.43-0.90.h5'))
models = [model_1, model_2, model_3, model_4]

In [10]:
label_list, correct_dict = test_model_ensemble(image_list)

  0%|          | 0/10970 [00:00<?, ?it/s]

Invalid image: /home/ssac21/UDIGO/dataset/test/수영장/img03_01_02_04.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/one-footed_skiing064.gif?type=w580
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/surf_board01.gif?type=w2
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf2698b4fe5dfcb1ac9187aa34c06699.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf42ca3213a39ecdf97ed5141b23bc71.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_580
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_001.gif?type=w1
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/095162b2-e684-4a65-a8de-48a93307a596.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/a4b6d053-877c-48db-a5a1-dc808ea6db46.gif
10961 data, test complete!
Test Accuracy: 0.92


In [11]:
count = collections.Counter(label_list)
accuracy_dict = {}

for label in correct_dict:
    accuracy_dict[label] = round(correct_dict[label]/count[label], 3)

sorted(accuracy_dict.items(), key=lambda x: x[1], reverse=True)

[('세빛섬', 1.0),
 ('청와대', 0.995),
 ('폭포', 0.995),
 ('국회의사당', 0.995),
 ('농구장', 0.99),
 ('호텔', 0.985),
 ('시장', 0.98),
 ('동물원', 0.98),
 ('동대문디자인플라자', 0.98),
 ('교회', 0.98),
 ('축구장', 0.98),
 ('스키장', 0.98),
 ('남산서울타워', 0.98),
 ('식물원', 0.975),
 ('PC방', 0.975),
 ('63빌딩', 0.975),
 ('지하철역', 0.97),
 ('패스트푸드점', 0.97),
 ('아쿠아리움', 0.965),
 ('컨벤션센터', 0.965),
 ('도서관', 0.965),
 ('캠핑장', 0.965),
 ('볼링장', 0.965),
 ('수상레포츠', 0.96),
 ('빨래방', 0.96),
 ('산', 0.955),
 ('야구장', 0.955),
 ('베이커리', 0.95),
 ('찜질방', 0.94),
 ('공연장', 0.938),
 ('헬스장', 0.935),
 ('궁궐', 0.935),
 ('미용실', 0.93),
 ('숲', 0.928),
 ('해수욕장', 0.925),
 ('수영장', 0.92),
 ('아이스링크', 0.919),
 ('쇼핑몰', 0.915),
 ('공항', 0.908),
 ('골프장', 0.895),
 ('놀이터', 0.89),
 ('성당', 0.89),
 ('롯데월드타워', 0.875),
 ('대학교', 0.85),
 ('키즈카페', 0.845),
 ('공원', 0.838),
 ('다리', 0.835),
 ('절', 0.82),
 ('한옥마을', 0.815),
 ('박물관', 0.811),
 ('미술관', 0.779),
 ('워터파크', 0.763),
 ('병원', 0.75),
 ('클럽', 0.73),
 ('놀이공원', 0.72)]